<a id="top"></a>
# Topics Outline

## [Why are 2 interacting RR robots different than a 5barlinkage?](#5bl_explanation)
## [What does the most general 5 bar linkage look like?](#g5bl_model)
 - derivation
 - gen5barlinkage class definition
 - gen5barlinkage exploration tool
 - discussion points
 
## [What does the human model look like?](#human_model)
- derivation
- human class definition
- human model exploration tool. 
- discussion points

## [Joint exploration - put the two together](#joint_exp)
- joint exploration tool 

## [Solve the constrained optimization problem](#solution)

# [notes](#notes)


<a id="5bl_explanation"></a>
# Why are 2 interacting RR robots different than a 5-bar-linkage? [ &#x21ea;](#top)



# Utility class [ &#x21ea;](#top)

In [1]:
import numpy as np

import warnings
warnings.filterwarnings('ignore')

class utils:
    """
    utility functions shared between different classes
    """
    @staticmethod
    def dist(x1,y1,x2,y2):
        """
        calculate the distance between 2 points
        """
        return ((x2 - x1)**2 + (y2 - y1)**2)**.5
    
    @staticmethod
    def mkCircle(x0,y0,r,n = 100):
        t = np.linspace(0,2* np.pi,n)
        x = r * np.cos(t) + x0
        y = r * np.sin(t) + y0
        return x,y

    @staticmethod
    def circIntersection(x1,y1,r1,x2,y2,r2):
        """characterize circle intersection"""
        dx = x2-x1; dy = y2-y1
        hyp = (dx**2 + dy**2)**.5
        if hyp > (r1 + r2):
            intersect = False
        else:
            intersect = True
        ipt = (x1 + (dx/2), y1 + (dy/2))
        ang = np.arctan2(dy,dx)
        return (ipt,ang,intersect)

    @staticmethod
    def withinIntersection(x1,y1,r1,x2,y2,r2,pt):
        """is the point contained within the intersection or two circles?"""
        def within(x,y,r,pt):
            dx = pt[0] - x ; dy = pt[1] - y
            hyp = (dx**2 + dy**2)**.5
            return r > hyp

        return (within(x1,y1,r1,pt) and within(x2,y2,r2,pt))
    
    @staticmethod
    def ROC(θ,upper=np.pi,lower=-np.pi):
        """
        rollover correction, to make sure the change in angle (Δθ) is accurately
        represented. the function can equally be used to find the minimal representation
        of an angle on the range (-π,π)
        """
        while (θ > upper) or (θ <  lower):
            if     θ >  upper: θ -= 2*np.pi
            elif   θ <  lower: θ += 2*np.pi
        return θ
    
    
    #notes on these angle helper functions: they all have the restriction of only working for the 
    #short way arround. if we want them to work for the long way arround, they need to be rethought
    # in the future. 
    @staticmethod
    def clampAng(ang,lower,upper):
        """
        clamp angle ang between the lower and upper angles. the implementation assumes that
        the distance between lower and upper is < π rad, and that the lower angle is "to the right"
        of the upper angle, in the context of the two angles being closer than π
        
        #note that this function doesn't operate sucessfully if lower and upper are switched, if 
         if the desired bounds between them are > np.pi
        """
        #rotate every angle so that lower is placed on the origin
        lower0 = lower 
        ang   -= lower ; upper -= lower ; lower = 0
        
        #represent all angles on (-π,π)
        ang = utils.ROC(ang) ; upper = utils.ROC(upper)
        
        #calc midline of upper and lower, and anti-midline m_bar
        midline = (upper + lower )/ 2
        m_bar = midline - np.pi
        
        #determine what angle to return, based on location
        if   ang >= lower and ang <= upper:  x = ang
        elif ang > upper:                    x = upper
        elif ang < m_bar:                    x = upper
        elif ang <= 0 and ang >= m_bar:      x = lower
        
        return x + lower0
    
    @staticmethod
    def between2Angles(a,b1,b2):
        """ is angle a between angles b1 and b2? the minimum angle"""
        #reduce angle to representation on -180 to 180
        angles= [a,b1,b2]
        angCorr = [utils.ROC(ang) for ang in angles]
        dists = [abs(utils.ROC(a-b1)),abs(utils.ROC(a-b2)),abs(utils.ROC(b1-b2))]
        if np.isclose(dists[2],dists[0] + dists[1]):
            return True
        return False
    
    #make this function opperate recursively when presented with a list? 
    @staticmethod
    def noNans(*args):
        for arg in args:
            if np.isnan(arg):
                return False
        return True
    
    @staticmethod
    def GD(x0,df,params):
        """
        peform gradient decent, given df, a function for calculating the 
        derivate of the function to be optimized. 
        """
        next_x = x0   # We start at x0
        gamma = 0.01  # Step size multiplier
        precision = 0.0001  # Desired precision of result
        max_iters = 10000  # Maximum number of iterations
        
        for _ in range(max_iters):
            current_x = next_x
            next_x = current_x - gamma * df(current_x,params)

            step = next_x - current_x
            if abs(step) <= precision:
                break

        return next_x
    
    def Rz(θ):
        """returns matrix which rotates by theta about positive z-axis"""
        return np.array([[ np.cos(θ), -np.sin(θ)],
                         [ np.sin(θ),  np.cos(θ)]])
    
    


In [ ]:
x = np.linspace(2*-np.pi,2*np.pi,500);
y = np.zeros(x.shape)
upper =  -np.pi/2;
lower =  -np.pi/4;
for i,xx in enumerate(x):
    ang = utils.clampAng(xx,lower,upper)
    y[i] = ang
    
_fig = go.Figure()
_fig.add_scatter(x=x,y=y)

<a id="g5bl_model"></a>
# What does the most general 5 bar linkage look like? [ &#x21ea;](#top)  

###  below we see a parametric representation of the kinematics of a general 5 bar linkage:

<img src="./imgs/general5bl_designParams.svg" style="width:1600px">

### * if we allow $L_{0} = 0$ and attach the end effector to $L_{2}$ with a decent sized $P_{x}$ extension segment, and place $(y_1,x_1)$ at the hip joint and $e_1$ at the knee, then we have the "exoskeleton" like design
### taken together, theses 11 variables represent the parameter space over which our optimization will ocure


The forward kinematics equations are: 

<font size="7">
$$
\begin{align*} 
X_{ee} &= L_{1l}C_{1l} + L_{2l}C_{12l} + L_{13l}C_{123l} + X_l \\ 
Y_{ee} &= L_{1l}S_{1l} + L_{2l}S_{12l} + L_{13l}S_{123l} + Y_l \\
X_{jt} &= L_{1l}S_{1l} + L_{2l}S_{12l} + X_l \\
X_{jt} &= L_{1l}S_{1l} + L_{2l}S_{12l} + Y_l \\
X_{jt} &= L_{1r}S_{1r} + L_{2r}S_{12r} + X_r \\
X_{jt} &= L_{1r}S_{1r} + L_{2r}S_{12r} + Y_r \\
\end{align*}
$$
</font>

* there are configurations for solving FK
* there are 4 configurations for solving IK
* there are 4 jacobians for the parellel robot, corresponding to each of the 4 methods. 
* i want to visualize how this effects the reciprical condition number, using our visualization techniques. 
* currently, configuration switching is not being considered in the analysis. 
    * recall that changing configurations requires passing through a singularity
* should we allow links to cross? - how would we deal with this, besides setting the value to zero when the links cross? ( assign a crossing penalty function) (0-1)
* direct penalty of singularity ()? or again should we only penalize indirectly? 
    * it seems to me that a singularity in the middle of the workspace is way worse than a singularity at then edge of the workspace. if it becomes a problem, we can invent a criteria   to elimitinate it, else carry on.

In [192]:
import plotly.graph_objs as go
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact , Layout , FloatSlider , Checkbox , Dropdown
from IPython.display import display
from numpy import linalg as LA
from shapely.geometry import Polygon
from IPython.core.debugger import set_trace


class rTauSpace():
    """define a torque space"""
    def __init__(self,t1max,t1min,t2max,t2min):
        self.t1max = t1max
        self.t1min = t1min
        self.t2max = t2max
        self.t2min = t2min
    def corners(self,repeat = False):
        """return the corners of the torque space polygon in CCW order from top right"""
        if repeat:
            cs = np.array([[self.t1max,self.t1min,self.t1min,self.t1max,self.t1max],
                           [self.t2max,self.t2max,self.t2min,self.t2min,self.t2max]])
        else:
            cs = np.array([[self.t1max,self.t1min,self.t1min,self.t1max],
                           [self.t2max,self.t2max,self.t2min,self.t2min]])
        return cs
    

class Gen5barlinkage:
    """
    this general 5 bar linkage class solves the kinematics, inverse kinematics, and 
    differential kinematics (jacobian) of a 2DOF, 5 bar linkage which is parametrized
    to be as general a 5 bar linkage as possible. 
    
    design parameters: 
        states that do not change during the course of the inner-loop of the optimization
        function, and represent design decisions to be made about the robot
    
    state variables: 
        angles that define the state of robot at a particular position
    """
    def __init__(self):
        #---continuous design parameters -----------
        self.xl =  0
        self.yl =  0
        self.xr = .5
        self.yr =  0
        
        self.L0  = .25      #calculated
        self.L1l = .25      #left link 1
        self.L2l = .25      #left link 2
        self.L3l = .25      #left link 3 
        self.L1r = .25      #right link 1
        self.L2r = .25      #right link 2
        
        self.θ3l =  .25      #angle parameter on [0-2π]
        
        #------discrete design paramters ---------
        
        #effecting the IK
        self.e1 = 'up'  #[up or down]
        self.ef = 'up'  # the state of the ficticious elbow, a proxy for e1
        self.e3 = 'up'
        #self.e2 = 'up' //determined by IK
        
        #---------- state variables -----------
        self.θ1l = None
        self.θ2l = None
        self.θ1r = None
        self.θ2r = None
        
        self.Xee = None  #required for θ driven kinematics
        self.Yee = None  #required for θ driven kinematics
        
        #---------- torque variables ----------
        self.τ1l = 100  #[N*m]
        self.τ1r = 100  #[N*m]
        self.τs = rTauSpace(self.τ1l,-self.τ1l,self.τ1r,-self.τ1r)
        
        
    def L0update(L0,θ0):
        """
        helper function for changing the internal state x2,y2 based on the polar 
        parameterization, (L0,θ0)
        """
        #FK for polar parameterization
        self.xr = self.xl + L0*np.cos(θ0);
        self.yr = self.xl + L0*np.sin(θ0);
    
    
    
    def _IK_RRR(self,x,y):
        """
        perform Inverse Kinematics on an RRR robot (constituting the left side the 
        5bar linkage in the reference position), in terms of x,y,θ3 this is an analytical
        solution to the IK problem, that assumes θ3 is locked and uses a fictutious link (l_f)
        approach. 
        """
        #define local vars: 
        L1 = self.L1l; L2 = self.L2l; L3 = self.L3l; θ3 = self.θ3l
        
        #determine the wrist state based on θ3  (consider refactoring into a property?)
        θ3 = utils.ROC(θ3,lower = 0,upper = 2*np.pi)
        if θ3 >= 0 and θ3 < np.pi:
            wrist = "down"
        if θ3 >= np.pi and θ3 <= 2*np.pi:
            wrist = "up"
    
        #determine Lf, the length of the ficticious segment
        a = np.pi - θ3
        Lf = (L2**2 + L3**2 - 2*(L2)*(L3)*np.cos(a))**.5
        
        #determine the IK for the RR robot formed by L1 and the fictitious Lf 
        θ1, θ2f = self._IK_RR(x,y,L1,Lf,self.ef)
        
        #determine the FK to the first link 
        x1 = L1*np.cos(θ1); y1 = L1*np.sin(θ1)
        
        #determine the IK from the tip of the first link, to the end effector location
        θ2, θ3 = self._IK_RR(x - x1,y - y1,L2,L3,wrist)
        θ2 -= θ1                             #angle correction for different coordinate frames 
        
        return θ1,θ2,θ3
    
    
        
    def _IK_RR(self,x,y,l1,l2,elbow):
        """
        perform Inverse Kinematics on an RR robot, see lect5 p.9-10 
        """
        #solve for intermediate values
        r = utils.dist(0,0,x,y) - 1e-12 #solves numerical issue at θ2 == 0 
        β = np.arccos((l1**2 + l2**2 - r**2) / (2*l1*l2))
        γ = np.arccos((r**2 + l1**2 - l2**2) / (2*r*l1))
        α = np.arctan2(y,x)
        
        #handle different elbow states
        θ1 = α - γ
        θ2 = np.pi - β
        
        if elbow == "down":
            return θ1,θ2
        elif elbow == "up":
            θ1pm = θ1 + 2*γ  
            θ2pm = -θ2   
            return θ1pm,θ2pm
        
    
    
    def _FK_RRR(self,θ1,θ2,θ3,l1,l2,l3):
        """
        forward kinematics for the an RRR robot
        """
        s1  = np.sin(θ1)      ; c1  = np.cos(θ1)
        s12 = np.sin(θ1 + θ2) ; c12 = np.cos(θ1 + θ2)
        s123 = np.sin(θ1 + θ2 + θ3) ; c123 = np.cos(θ1 + θ2 + θ3)
    
        #forward kinematics
        origin = [0,0]
        l1_tip = [l1*c1 , l1*s1]                                     
        l2_tip = [l1_tip[0] + l2*c12, l1_tip[1] + l2*s12]
        l3_tip = [l2_tip[0] + l3*c123, l2_tip[1] + l3*s123]
        
        #reorganize into vectors
        x = [origin[0],l1_tip[0],l2_tip[0],l3_tip[0]]
        y = [origin[1],l1_tip[1],l2_tip[1],l3_tip[1]]
        
        return x,y
    
    
    
    def _FK_RR(self,θ1,θ2,l1,l2):
        """
        forward kinematics for RR robot
        """
        x,y = self._FK_RRR(θ1,θ2,0,l1,l2,0)
        return x[:-1],y[:-1]
    
    
    
    def IK5bl(self,x,y):
        """
        calculate the inverse kinematics for the entire 5-bar-linkage robot. 
        """
        #see if the left RRR robot can reach the target point with the end effector
        θ1l,θ2l,θ3l = self._IK_RRR(x - self.xl,y - self.yl)
        
        #calculate the position of joint e2
        xs,ys = self._FK_RR(θ1l,θ2l,self.L1l,self.L2l)                    #local to RRR link
        xe2 = xs[2] + self.xl; ye2 = ys[2] + self.yl                      #global frame
        
        #calculate the IK for tip of the right RR robot to the point E2
        θ1r,θ2r = self._IK_RR(xe2 - self.xr,ye2 - self.yr,self.L1r,self.L2r,self.e3)
        
        #update internal representation of state
        self.θ1l,self.θ2l,self.θ1r,self.θ2r  =  θ1l,θ2l,θ1r,θ2r 
        self.Xee = x; self.Yee = y
        
        return θ1l,θ2l,self.θ3l,θ1r,θ2r 
    
    
    
    def θdrivenKinematics5bl(self,θ1l,θ1r):
        """
        given an initial system configuration (which is taken to be the internal state)
        numerically solve the system when either θ1 or θ2 are driven by some (small) amount. 
        this is to get an sense for what the FK will look like when driving via the actuators
        """
        #assume at least 1 call has been made to IK5bl, and therefore the system has been initialized
        q = np.array([self.Xee,self.Yee,self.θ2l,self.θ2r])
        self.θ1l = θ1l; self.θ1r = θ1r  #update state with targets
        
        def Φ(q):
            """
            system of non-linear equations constituting the FK of the gen5bl
            """
            #extract variables from q
            Xee = q[0]; Yee = q[1]
            θ2l = q[2]; θ2r = q[3] 
            
            #static values
            θ1l = self.θ1l ; θ1r = self.θ1r ; θ3l = self.θ3l 
            
            #forward kinematics to the end effector
            xs,ys = self._FK_RRR(θ1l,θ2l,θ3l,self.L1l,self.L2l,self.L3l)
            eq1 = xs[3] + self.xl - Xee
            eq2 = ys[3] + self.yl - Yee
            
            #left and right robots meet at a joint
            xls , yls = self._FK_RR(θ1l,θ2l,self.L1l,self.L2l)
            xrs , yrs = self._FK_RR(θ1r,θ2r,self.L1r,self.L2r)
            eq3 = xls[2] + self.xl - xrs[2] - self.xr
            eq4 = yls[2] + self.yl - yrs[2] - self.yr
            
            ϕ = np.array([[eq1],[eq2],[eq3],[eq4]])
            return ϕ
        
        
        def Φ_q(q):
            """
            derivative of Φ w.r.t the variables q
            """
            #extract variables from q
            Xee = q[0]; Yee = q[1]
            θ2l = q[2]; θ2r = q[3] 
            
            #static values
            θ1l = self.θ1l ; θ1r = self.θ1r ; θ3l = self.θ3l  
            
            #compute sines and cosines
            s12l  = np.sin(θ1l + θ2l)        ; c12l  = np.cos(θ1l + θ2l)
            s123l = np.sin(θ1l + θ2l + θ3l)  ; c123l = np.cos(θ1l + θ2l + θ3l)
            s12r  = np.sin(θ1r + θ2r)        ; c12r  = np.cos(θ1r + θ2r)
            
            #form the jacobian explicitly
            #                Xee   Yee                 θ2l                       θ2r
            ϕ_q = np.array([[-1,    0,   -self.L2l*s12l - self.L3l*s123l,         0     ],   #eq1
                            [ 0,   -1,    self.L2l*c12l + self.L3l*c123l,         0     ],   #eq2
                            [ 0,    0,   -self.L2l*s12l,                  self.L2r*s12r ],   #eq3
                            [ 0,    0,    self.L2l*c12l,                 -self.L2r*c12r ]])  #eq4
            
            return ϕ_q
            
            
        #perform Newton-Rhapson
        Δq = np.ones([4])
        ϵ = 10**-7 
        maxIter = 100
        n = 0
        while np.linalg.norm(Δq) > ϵ:
            #print(Φ_q(q))
            Δq = np.linalg.solve(Φ_q(q), -Φ(q))
            q = q + Δq.squeeze()
            n +=1
            if n > maxIter:
                break
        
        #set state variables (for next time around) and return
        self.Xee = q[0] ; self.Yee = q[1]; self.θ2l = q[2]; self.θ2r = q[3]
        return self.θ1l,self.θ2l,self.θ3l,self.θ1r,self.θ2r  #for FK plot
        
        
    
    def FKplot5bl(self,θ1l,θ2l,θ3l,θ1r,θ2r):
        """
        given the joint angles, what are the points to plot to view the robot?
        return the points as 3 objects so they can be plotted in different styles
        """
        xs,ys = self._FK_RRR(θ1l,θ2l,θ3l,self.L1l,self.L2l,self.L3l)        #local to RRR link
        xs = self.xl + np.array(xs) ; ys = self.yl + np.array(ys)           #global frame
        
        left        = (xs,ys)
        
        xs,ys = self._FK_RR(θ1r,θ2r,self.L1r,self.L2r)                      #local frame
        xs = self.xr + np.array(xs) ; ys = self.yr + np.array(ys)           #global frame
        
        right = (xs,ys)
        
        return left,right
        
        
        
    def Jac(self,x,y,mode = "force"):
        """
        calculate the jacobian matrix at position x,y, given the elbow states. 
        """
        #calculate the IK for the given state of the elbows, and x,y
        θ1l,θ2l,θ3l,θ1r,θ2r  =  self.IK5bl(x,y)
        
        #define sine's and cosines, lengths
        c123l =  np.cos(θ1l + θ2l + θ3l) ; s123l = np.sin(θ1l + θ2l + θ3l)
        c1r   =  np.cos(θ1r)             ; s1r   = np.sin(θ1r) 
        c12r  =  np.cos(θ1r + θ2r)       ; s12r  = np.sin(θ1r + θ2r)
        L3l = self.L3l; L1r = self.L1r; L2r = self.L2r
        
        #calculate the force jacobian 
        #                  x(θ's)                  y(θ's)  
        jac = np.array([[-L3l*s123l          , L3l*c123l        ],      #θ1l
                        [-L1r*s1r - L2r*s12r , L1r*c1r + L2r*c12r]])    #θ1r
        
        if mode == "force":
            return jac
        elif mode == "velocity":
            return jac.T
    
    def J_invT(self,x,y):
        """
        calculate the Inverse Transpose Jacobian which is used to map joint torques [τ1l, τ1r]
        to end effector forces, using the method of finite differences from the analytical IK
        """
        
        #calculate the sensitivity of IK to changes in x and y with finite differences
        δ = .0005
        #col 1
        θ1l_p,__,__,θ1r_p,__  =  self.IK5bl(x+δ,y)
        θ1l_m,__,__,θ1r_m,__  =  self.IK5bl(x-δ,y)
        δθ11 = θ1l_p - θ1l_m
        δθ21 = θ1r_p - θ1r_m
        
        #col 2
        θ1l_p,__,__,θ1r_p,__  =  self.IK5bl(x,y+δ)
        θ1l_m,__,__,θ1r_m,__  =  self.IK5bl(x,y-δ)
        δθ12 = θ1l_p - θ1l_m
        δθ22 = θ1r_p - θ1r_m
        
        #assemble in inverse jacobian
        δx = 2*δ ; δy = 2*δ
        #                        x       y
        J_inv = np.array([[δθ11/δx , δθ12/δy ],    #θ1l 
                          [δθ21/δx , δθ22/δy]])    #θ1r
        
        
        return J_inv.T
        
    def J_vel_analytical(self,x,y,jac = "J"):
        """
        using the method of velocity loop closure, jacobians were determined analytically
        for the Gen5bl system under the following assumptions:
        x = [Xee,Yee,ϕ]
        θ = [θ1l,θ1r]
        """
        
        
        if jac == "Jq":
            pass
        if jac == "Jx":
            pass
        
        if Jac == "J":
            pass
        
        
    def bounds(self,leftTrimmed = False):
        """
        using an analytical approach, find the boundry where it's possible to solve IK.
        the boundry in space where IK is solveable is defined by where the end effector is
        when one of the two grounded serial robots is at full extension. for the left robot,
        this is simply a circle. for the right robot, its a more complicated shape.
        """
        #calculate the effective radii of the left and right robots
        a = np.pi - self.θ3l
        Lf = (self.L2l**2 + self.L3l**2 - 2*(self.L2l)*(self.L3l)*np.cos(a))**.5
        Rl = self.L1l + Lf
        Rr = self.L1r + self.L2r
        
        #make a set of points for the circular boundries of each arm
        xl = self.xl ; yl = self.yl; xr = self.xr ; yr = self.yr
        ϵ = .0001; #need a little numerical cushion to get all the points
        Lbx,Lby = utils.mkCircle(xl,yl,Rl - ϵ)
        Rx,Ry = utils.mkCircle(xr,yr,Rr - ϵ)
        
        #only include the left bondry points that can be reached through gen5bl IK
        if leftTrimmed:
            _Lbx = [] ; _Lby = []
            for i,x in enumerate(Lbx):
                θ1l,θ2l,θ3l,θ1r,θ2r = self.IK5bl(Lbx[i] - xl,Lby[i] - yl)
                if not np.isnan(θ1r) and not np.isnan(θ2r): 
                    _Lbx.append(Lbx[i])
                    _Lby.append(Lby[i])       
            Lbx = _Lbx; Lby = _Lby
        
       #for right robot full extension, perform IK to joint e2, then FK on Left side
        Rbx = [] ; Rby = []
        for i,x in enumerate(Rx):
            θ1,θ2 = self._IK_RR(Rx[i] - xl,Ry[i] - yl,self.L1l,self.L2l,self.ef)
            if  not np.isnan(θ1) and not np.isnan(θ2):
                xs,ys = self._FK_RRR(θ1,θ2,self.θ3l,self.L1l,self.L2l,self.L3l)
                x = xs[3] + xl ; y = ys[3] + yl
                Rbx.append(x)
                Rby.append(y)
            
        return (Lbx,Lby),(Rbx,Rby) 
                
                
    def boundingBox(self):
        """
        calculate the cartesian bounding box where solving IK is feasable
        output: xmin,xmax,ymin,ymax
        """
        ls,rs = self.bounds(leftTrimmed = True)
        xs = np.hstack([ls[0],rs[0]])
        ys = np.hstack([ls[1],rs[1]])
        δ = .1
        xmin = np.min(xs)-δ ; xmax = np.max(xs)+δ
        ymin = np.min(ys)-δ ; ymax = np.max(ys)+δ        
        return xmin,xmax,ymin,ymax
    
    
    def condGrid(self,nx = 20,ny = 20,clean=False):
        """
        evaluate the reciprical condition number on a grid within the reachable
        space of the gen5bl robot
        """
        #setup sampling vectors
        xmin,xmax,ymin,ymax = self.boundingBox()
        X = np.linspace(xmin,xmax,nx)
        Y = np.linspace(ymin,ymax,ny)
    
        #evaluate reciprical condition number
        Z = np.zeros((X.shape[0],Y.shape[0]))
        Z = np.full_like(Z, np.nan)
        if clean: return X,Y,np.full_like(Z,np.nan)
        for i,x in enumerate(X):
            for j,y in enumerate(Y):
                invCond = 1/np.linalg.cond(self.J_invT(x,y))
                Z[i,j] = invCond
        return X,Y,Z.T
    
    
    def FSplot(self,x,y,Scale):
        """
        given a robot configuration, return the xs and ys for plotting
        the force parellelogram
        """
        cs = (self.J_invT(x,y) @  self.τs.corners(repeat=True)) * Scale
        xs = cs[0,:] ; ys = cs[1,:]
        return xs,ys
    
        
        
        
#---------------------------------- plotting ---------------------------------
R = Gen5barlinkage()

#setup contour plot for condition number 
#colorscale = [[0, 'green'],[.2,'royalblue'],[1, 'blue']]
colorscale = [[0, 'green'],[1, 'blue']]
X,Y,Z = R.condGrid()
_fig = go.Figure(data = go.Contour(z=Z,x=X,y=Y,colorscale=colorscale,contours_coloring='heatmap'))
Rfig = go.FigureWidget(_fig)
#Rfig = go.FigureWidget()

#configure layout of graph
Rfig.update_xaxes(range=[-1,1])
Rfig.update_yaxes(range=[-1,1])

side = 800
Rfig.update_layout(
    autosize=False,
    width=side + 100,
    height=side,
    legend=dict(x=.025, y=.975),
    margin=dict(l=80, r=80, t=0, b=80))


#add condition number annotation
Rfig.add_annotation(
        x=-.78,
        y=.6,
        xref="x",
        yref="y",
        text="",
        font=dict(
            family="Courier New, monospace",
            size=16,
            color="#ffffff"
            ),
        align="center",
        ax=0,
        ay=0,
        bordercolor="#c7c7c7",
        borderwidth=2,
        borderpad=4,
        bgcolor="#ff7f0e",
        opacity=0.8
        )

#initialize all plots present in visualization
Rfig.add_scatter(line=dict(color='royalblue'), name = "left")
Rfig.add_scatter(line=dict(color='red'), name = "right")
Rfig.add_scatter(mode='markers', marker=dict(size=12,color="black"), name = "End Effector")
Rfig.add_scatter(line=dict(dash='dash',color='royalblue'), name = "RRR boundry")
Rfig.add_scatter(line=dict(dash='dash',color='red'), name = "RR boundry")
Rfig.add_scatter(fill='tozeroy',fillcolor='rgba(225, 0, 0, .2)',mode= 'none'  ,name="robot forceSpace")



#setup UI
layout=Layout(width='500px', height='20px')
layout1=Layout(width='200px', height='30px')
layout2=Layout(width='350px', height='30px')
lmin = .1 ; lmax = .75; lval = .25
rng = 1; step = .001
N = 0


#modes
ef     = Dropdown(options=["up","down"],value="up",description='ef',layout=layout1)
e3     = Dropdown(options=["up","down"],value="up",description='e3',layout=layout1)
mode   = Dropdown(options=["None","boundry","cond"],value="None",description='Mode',layout=layout1)
drive  = Dropdown(options=["xy","θ"],value="xy",description='drive',layout=layout1)
fsView = Checkbox(value=False,description="fsView",dindent=False,layout=layout1)


#params
L1l = FloatSlider(min=lmin,max=lmax,step=step,value=lval,description='L1_l', layout=layout2)
L2l = FloatSlider(min=lmin,max=lmax,step=step,value=lval,description='L2_l', layout=layout2)
L3l = FloatSlider(min=0   ,max=lmax,step=step,value=lval,description='L3_l', layout=layout2)

L1r = FloatSlider(min=lmin,max=lmax,step=step,value=lval,description='L1_r', layout=layout2)
L2r = FloatSlider(min=lmin,max=lmax,step=step,value=lval,description='L2_r', layout=layout2)
θ3l = FloatSlider(min=-np.pi,max=np.pi,step=step,value=lval,description='θ3_l', layout=layout2)

xl = FloatSlider(min=-rng,max=rng,step=step,value=.0,description='x_l', layout=layout)
xr = FloatSlider(min=-rng,max=rng,step=step,value=.5,description='x_r', layout=layout)

yl = FloatSlider(min=-rng,max=rng,step=step,value=.0,description='y_l', layout=layout)
yr = FloatSlider(min=-rng,max=rng,step=step,value=.0,description='y_r', layout=layout)

#drivers
θ1l = FloatSlider(min=-np.pi,max=np.pi,step=step,value=.0,description='θ1_l', layout=layout)
xee = FloatSlider(min=-rng,max=rng,step=step,value=.5,description='xee', layout=layout)

θ1r = FloatSlider(min=-np.pi,max=np.pi,step=step,value=.0,description='θ1_r', layout=layout)
yee = FloatSlider(min=-rng,max=rng,step=step,value=.5,description='yee', layout=layout)

#assemble UI
row1 = widgets.HBox([ef,e3,mode,drive,fsView])
row2 = widgets.HBox([L1l,L2l,L3l])
row3 = widgets.HBox([L1r,L2r,θ3l])
row4 = widgets.HBox([xl,xr])
row5 = widgets.HBox([yl,yr])
row6 = widgets.HBox([θ1l,xee])
row7 = widgets.HBox([θ1r,yee])

ui = widgets.VBox([row1,row2,row3,row4,row5,row6,row7])


#setup update function
wdict = {"ef":ef,"e3":e3,"mode":mode,"drive":drive,"fsView":fsView,
         "L1l":L1l,"L2l":L2l,"L3l":L3l,
         "L1r":L1r,"L2r":L2r,"θ3l":θ3l,
         "xl":xl,"xr":xr,
         "yl":yl,"yr":yr,
         "_θ1l":θ1l,"_xee":xee,
         "_θ1r":θ1r,"_yee":yee}


def update(ef="up",e3="up",mode="None",drive="xy",fsView=False,
           L1l=1,L2l=1,L3l=1,
           L1r=1,L2r=1,θ3l=1,
           xl=0,xr=1,
           yl=0,yr=1,
           _θ1l=1,_xee=1,    #use _ so we have access to the slider objects
           _θ1r=1,_yee=1):
    
    with Rfig.batch_update():
        
        #update parameters
        R.ef = ef ; R.e3 = e3
        R.L1l = L1l ; R.L2l=L2l ; R.L3l = L3l
        R.L1r = L1r ; R.L2r=L2r ; R.θ3l = θ3l
        R.xl = xl ; R.xr = xr
        R.yl = yl ; R.yr = yr
        
        #calculate the robot's configuration
        if drive == "xy":
            _θ1l,_θ2l,_θ3l,_θ1r,_θ2r = R.IK5bl(_xee,_yee)
            left,right = R.FKplot5bl(_θ1l,_θ2l,_θ3l,_θ1r,_θ2r)
            
            #keep θ's up to date for solving numberical IK
            if utils.noNans(_θ1l,_θ1r):
                global θ1l ; global θ1r
                θ1l.value = utils.ROC(_θ1l); θ1r.value = utils.ROC(_θ1r)
            
        if drive == "θ":
            _θ1l,_θ2l,_θ3l,_θ1r,_θ2r = R.θdrivenKinematics5bl(_θ1l,_θ1r)
            left,right = R.FKplot5bl(_θ1l,_θ2l,_θ3l,_θ1r,_θ2r)
            
        #draw the robot (if solution exists)
        Rfig.data[1]['x'] = left[0]
        Rfig.data[1]['y'] = left[1]

        Rfig.data[2]['x'] = right[0]
        Rfig.data[2]['y'] = right[1]  

        #plot a dot at the end effector location
        Rfig.data[3]['x'] = [_xee]
        Rfig.data[3]['y'] = [_yee]
        
        #annotate the condition number of the current configuration
        invCond = 1/np.linalg.cond(R.J_invT(_xee,_yee))
        Rfig.layout['annotations'][0]['text'] = "InvCond=" + str(round(invCond,3))
        
        #draw the force space 
        if fsView:
            scale = .0005
            xs,ys = R.FSplot(_xee,_yee,scale)
            Rfig.data[6]['x'] = xs + _xee
            Rfig.data[6]['y'] = ys + _yee
        
        else:
            Rfig.data[6]['x'] = []
            Rfig.data[6]['y'] = []
            
            
            
            
        #draw the reachable boundry
        if mode == "boundry" or mode == "cond":
            ls,rs = R.bounds()
            Rfig.data[4]['x'] = ls[0]
            Rfig.data[4]['y'] = ls[1]
    
            Rfig.data[5]['x'] = rs[0]
            Rfig.data[5]['y'] = rs[1]
        
        
        #draw a contour plot of the reciprical condition number 
        if mode == "cond":
            X,Y,Z = R.condGrid()
            Rfig.data[0]['x'] = X
            Rfig.data[0]['y'] = Y
            Rfig.data[0]['z'] = Z  
            
        #clean up 
        if mode == "None":
            #clean up contour plot (this nukes R state)
            X,Y,Z = R.condGrid(clean=True)
            Rfig.data[0]['x'] = X
            Rfig.data[0]['y'] = Y
            Rfig.data[0]['z'] = Z
            
            #clean up boundry
            Rfig.data[4]['x'] = []
            Rfig.data[4]['y'] = []
            
            Rfig.data[5]['x'] = []
            Rfig.data[5]['y'] = []
            
        #reset R state properly
        R.IK5bl(_xee,_yee)
        
        
        
#display interactive figure
out = widgets.interactive_output(update, wdict)
display(ui)
Rfig


FigureWidget({
    'data': [{'colorscale': [[0, 'green'], [1, 'blue']],
              'contours': {'coloring':…

In [156]:
out

Output()

### high definition condition number portrait

In [3]:
colorscale = [[0, 'green'],[1, 'blue']]
X,Y,Z = R.condGrid(nx=300,ny=300)
HD_fig = go.Figure(data = go.Contour(z=Z,x=X,y=Y,colorscale=colorscale,contours_coloring='heatmap'))
HD_fig

#config layout
side = 800
HD_fig.update_layout(
    autosize=False,
    width=side + 100,
    height=side,
    legend=dict(x=.025, y=.975),
    margin=dict(l=80, r=80, t=0, b=80))

#configure layout of graph
HD_fig.update_xaxes(range=[-1,1])
HD_fig.update_yaxes(range=[-1,1])

import plotly.graph_objects as go
img_rgb = [[[255, 0, 0], [0, 255, 0], [0, 0, 255]],
           [[0, 255, 0], [0, 0, 255], [255, 0, 0]]]
fig = go.Figure(go.Image(Z))
fig.show()

NameError: name 'HD_fig' is not defined

### Discussion Points:
* joint limits on $\theta_1$ and $\theta_2$ or joint impingement? 
* link crossing? 
* ground plane? 
* $L_0$ constraint? 



to do: 
* draw Ef in the robot parameterization figure
* make a contour plot of the inverse condition number to check it's integrety
* make a numerical solver for running the system in $\theta_1, \theta_2$ drive mode
* add the code to compute the force trapizoid
* view the force trapezoid when fsview is selected





what are our objectives with this visualizer? 
* ensure that everything we have done is correct, and there are no errors, especially in the calculation of the IK and the jacobian. 
* evaluate the effects of changing different parameters on outputs like 
    *the reachable workspace
    *the reciprical condition number of the jacobian. 
* therefore, we want 
* want a way to drive the robot either with x and y (relative to drive it with theta's)
* view the force trapezoid
* view the inverse condition number. 
* adjust the states of the elbows. 


# references
- calculation of the jacobian using velocity methods - lung-wen tsai


<a id="human_model"></a>
# what does the human model look like? [ &#x21ea;](#top)

<img src="./imgs/humanModel.svg" style="width:1700px">


### Discussion Points:
### IK - approaches to making IK realistic
* define springs and minimize a potential energy function
* develop a huristic for $ϕ(x,y)$
* bounds coupling


## to do:
(saturday) 
 - have fun, be creative, speed. 
 - write jacobian function X
 - implement hard limits on the joints using repulsion functions
 - tune linear model (figureing out why x0 isn't working exactly as you anticipated) so that it's good over the desired workspace
 - develop, train, store and compare the surigate model
 - finish the leg model visual
 
(sunday)
 - develop a first principles based bounds function which is quick to evaluate
 - develop and tune spline based torque classes (from literature / opensim?) 
 - implement mapping forward through the jacobian into the force space
 - visualize the force space, bounds, IK, all together -> the human model is good to go. 
 
 

## references:

<!-- $$
\begin{align}
τ_{h} &= f(θ_{h},θ_{k}) \\
τ_{k} &= f(θ_{h},θ_{k},θ_{a}) \\
τ_{a} &= f(θ_{k},θ_{a})
\end{align}
$$ -->

In [227]:
cs = np.ones((3,8))
cs
x = [cs[:,1],cs[:,2]]
np.array(x)

array([[1., 1., 1.],
       [1., 1., 1.]])

In [350]:
import plotly.graph_objs as go
import numpy as np
import ipywidgets as widgets
from ipywidgets import interact , Layout , FloatSlider , Checkbox , Dropdown
from IPython.display import display
from numpy import linalg as LA
from shapely.geometry import Polygon
from scipy.spatial import ConvexHull
from scipy.optimize import minimize

    
##design decision - should I treat Human more functionally - so basically as a static class, but without state, and more of a container for functions, or should I rely heavily 
#on state caching, which means I have to guarentee order of function calls inorder for things to be up to date. this is the fundemental tradeoff of functional vs stateful approaches. 
# other design decisions, would sampling happen in cartesian, or in joint space for the optimization? - at least some of that sampling should live in another class that handles the
#optimization between the human and the gen5bl



#------------------------------------------------ helper classes -----------------------------------------------------
class Foot:
    """
    a physiologic(-ish) foot segment for the human model
    """
    def __init__(self,footLength):
        #compute the critical points defining the foot geometry
        self.jointOrigin        = np.array([0,0])           #apex of the triangle
        self.footLength         = footLength                #forms the base of the triangle
        self.Hindfoot           = .23*self.footLength        #foot is separated into hindfoot and
        self.forfoot            = footLength-self.Hindfoot  #forfoot
        self.footHeight         = .2*self.footLength        #distance from joint origin to line perpendicular to foot length
        self.EEattachment       = .45*self.footLength       #point where the robot and human attach, the End Effector
        self.effectiveMomentArm = np.sqrt(self.footHeight**2 + self.EEattachment**2)
        
        #publically exposed variables:
        self.len = self.effectiveMomentArm
        self.α = np.arctan(self.EEattachment/self.footHeight) #angle offset from height 
    
    def PlotPts(self,θ):
        """returns points to plot the Foot"""
        #define points
        origin  = self.jointOrigin
        midfoot = np.array([0,-self.footHeight])
        heel    = np.array([-self.Hindfoot,-self.footHeight])
        toe     = np.array([self.forfoot,-self.footHeight])
        EE      = np.array([self.EEattachment,-self.footHeight])
        
        #assemble points for continuous line plot
        pts = np.hstack((midfoot[np.newaxis].T,
                         origin [np.newaxis].T,
                         heel   [np.newaxis].T,
                         toe    [np.newaxis].T,
                         origin [np.newaxis].T,
                         EE     [np.newaxis].T))
        
        #static transform to allign ee appropriately for visualization,
        #doesn't impact jacobian
        β = ((np.pi/2) - self.α)  
        return utils.Rz(θ + β) @ pts 
    
    
class Hip:
    """
    hip class handles the calculation of hip joint torque, and simple joint limits.
    """
    def __init__(self):
        self.θmax = np.pi/180 *  70
        self.θmin = np.pi/180 * -90
        
        #define state information for spline fit here
        #self.θ_offset = 0  #why do we need this?

        #define state information for spline fit here
        self.simple = True  #used for debugging
        self._τE =  -1 
        self._τF =   1

    def τE(self,θh,θk,θa):
        return self._τE
    
    def τF(self,θh,θk,θa):
        return self._τF
        
        
class Knee:
    """
    knee class handles the calculation of knee joint torque, and simple joint limits
    """
    def __init__(self):
        self.θmax = np.pi/180 *  0
        self.θmin = np.pi/180 * -120
        
        #define state information for spline fit here
        self.simple = True  #used for debugging
        self._τE =  -1 
        self._τF =   1

    def τE(self,θh,θk,θa):
        return self._τE
    
    def τF(self,θh,θk,θa):
        return self._τF
        

class Ankle:
    """
    Ankle class handles the calculation of Ankle joint torque, and simple joint limits
    """
    def __init__(self,foot):
        self.foot = foot
        
        #limits from foot flat
        self.θmax = np.pi/180 *  20  + self.foot.α   #dorsi-flexion
        self.θmin = np.pi/180 * -50  + self.foot.α   #plantar-flexion
    
        #define state information for spline fit here
        self.simple = True  #used for debugging
        self._τE =  -1 
        self._τF =   1

    def τE(self,θh,θk,θa):
        return self._τE
    
    def τF(self,θh,θk,θa):
        return self._τF

class hTauSpace:
    """
    class for managing the torque space of the human model
    """
    def __init__(self,hip,knee,ankle):
        self.hip = hip
        self.knee = knee
        self.ankle = ankle
       
    def corners(self,θh,θk,θa):
        """return the corners of the 3 dimensional torque space rectangular prism"""
        
        #order from diagram - hardcoded
        cs = np.array([[self.hip.τF(θh,θk,θa),self.knee.τF(θh,θk,θa),self.ankle.τF(θh,θk,θa)],  #v0
                       [self.hip.τF(θh,θk,θa),self.knee.τE(θh,θk,θa),self.ankle.τF(θh,θk,θa)],  #v1
                       [self.hip.τE(θh,θk,θa),self.knee.τF(θh,θk,θa),self.ankle.τF(θh,θk,θa)],  #v2
                       [self.hip.τF(θh,θk,θa),self.knee.τF(θh,θk,θa),self.ankle.τE(θh,θk,θa)],  #v3
                       [self.hip.τF(θh,θk,θa),self.knee.τE(θh,θk,θa),self.ankle.τE(θh,θk,θa)],  #v4
                       [self.hip.τE(θh,θk,θa),self.knee.τE(θh,θk,θa),self.ankle.τF(θh,θk,θa)],  #v5
                       [self.hip.τE(θh,θk,θa),self.knee.τF(θh,θk,θa),self.ankle.τE(θh,θk,θa)],  #v6
                       [self.hip.τE(θh,θk,θa),self.knee.τE(θh,θk,θa),self.ankle.τE(θh,θk,θa)]]) #v7
        return cs.T
    
    
    def planeCut(self,θh,θk,θa,n_p):
        """
        given a normal vector n_p defined by (nx,ny,nz) and a torque space rectangular prism calculated
        from (θh,θk,θa) - calculate the verticies of the intersection polygon
        """
        #calculate the corners of the torque space rectangular prism
        cs = self.corners(θh,θk,θa)
        n_p = n_p.flatten()
        
        #systematically enumerate all rectangular prism edges. 
        edges = ["e01","e02","e03","e14","e15","e25","e26","e34","e36","e47","e57","e67"]
        
        #calculate the vertecies of intersection between the zero-torque plane and the 12 prism edges
        verts = []
        for vi in range(0,8):
            for vj in range(0,8):
                edge = "e" + str(vi) + str(vj)
                if edge in edges:
                    #find the intersection point λ between plane and edge eij
                    d = 0                            #zero-torque plane passes through origin. 
                    Vi = cs[:,vi]                    #ith corner vertex
                    eij = cs[:,vj] - cs[:,vi]        #vector defining edge ij
                    
                    #make sure edge doesn't lay in the plane n_p
                    if abs(np.dot(n_p,eij)) < .000005:
                        pass
                    else:
                        λ = (d - np.dot(n_p,Vi)) / (np.dot(n_p,eij))
                    
                        #determine if intersection happens inside the prism
                        if λ >= 0 and λ <= 1:
                            verts.append(Vi + λ * eij)
        
        return np.array(verts)  #[npts,3]
    
    def n_p(self,jac_t):
        """
        calculate the direction of the normal vector in tau space, from the force jacobian 
        """
        n_p = jac_t @ np.array([[0],[0],[1]])
        return n_p  #because of the structure of jac, this is always [1,1,1].T
        
    
    def prismPlot(self,θh,θk,θa):
        """return points for plotting torque-space rectangular prism"""
        cs = self.corners(θh,θk,θa)
        pts = np.vstack((cs[:,0],cs[:,1],cs[:,4],cs[:,7],
                         cs[:,7],cs[:,6],cs[:,3],cs[:,0],
                         cs[:,0],cs[:,2],cs[:,5],cs[:,7],
                         cs[:,7],cs[:,4],cs[:,3],cs[:,0],
                         cs[:,0],cs[:,1],cs[:,5],cs[:,7],
                         cs[:,7],cs[:,6],cs[:,2],cs[:,0],))
        
        return pts[:,0], pts[:,1], pts[:,2] 
        
    
    def polygonPlot(self,θh,θk,θa,n_p):
        """
        return plots for vertecies of 
        """
        verts = self.planeCut(θh,θk,θa,n_p)
        #hull = ConvexHull(verts)
        #verts = verts[hull.vertices,:]
        return verts[:,0] , verts[:,1], verts[:,2]
    
    
    def n_pplot(self):
        pass
    
    def θhPlanePlot(self):
        pts = np.array([[0,self.knee.τF(θh,θk,θa),self.ankle.τF(θh,θk,θa)],
                        [0,self.knee.τF(θh,θk,θa),self.ankle.τE(θh,θk,θa)],
                        [0,self.knee.τE(θh,θk,θa),self.ankle.τF(θh,θk,θa)],
                        [0,self.knee.τE(θh,θk,θa),self.ankle.τE(θh,θk,θa)]]) 
        return pts[:,0], pts[:,1], pts[:,2] 
    
    def θkPlanePlot(self):
        pts = np.array([[self.hip.τF(θh,θk,θa),0,self.ankle.τF(θh,θk,θa)],
                        [self.hip.τF(θh,θk,θa),0,self.ankle.τE(θh,θk,θa)],
                        [self.hip.τE(θh,θk,θa),0,self.ankle.τF(θh,θk,θa)],
                        [self.hip.τE(θh,θk,θa),0,self.ankle.τE(θh,θk,θa)]]) 
        return pts[:,0], pts[:,1], pts[:,2] 
    
    def θaPlanePlot(self):
        pts = np.array([[self.hip.τF(θh,θk,θa),self.knee.τF(θh,θk,θa),0],
                         [self.hip.τF(θh,θk,θa),self.knee.τE(θh,θk,θa),0],
                         [self.hip.τE(θh,θk,θa),self.knee.τF(θh,θk,θa),0],
                         [self.hip.τE(θh,θk,θa),self.knee.τE(θh,θk,θa),0]])
        return pts[:,0], pts[:,1], pts[:,2]  
    
#------------------------------------------------ Human -----------------------------------------------------    

class Human:
    def __init__(self):
        #segment lengths
        self.femur      = .4075    #[meters - from anthopometric tables] 
        self.shank      = .390     #[meters] 
        self.footLength = .255     #[meters]
        self.foot = Foot(self.footLength)
        self.reach = self.femur + self.shank + self.foot.len
        
        #joints and torque space
        self.hip = Hip()
        self.knee = Knee()
        self.ankle = Ankle(self.foot)
        self.τs = hTauSpace(self.hip,self.knee,self.ankle)
        
        #complex limits
        self.αMin = np.pi/180 * -75
        self.αMax = np.pi/180 *  0
        
        #tuning vars for Jch
        self.h0 = -.50; self.m_h = -.8
        self.k0 = -1.79; self.m_k = -1.0; self.ang = 0

        #state:
        self.θh = 0
        self.θk = 0
        self.θa = 0 
        
        #caches
        self.samplePoints = []  
        self.fsCache = dict()    

        
    #------------- private functions --------------------
    def _IK_RR(self,x,y,l1,l2,elbow):
        """
        perform Inverse Kinematics on an RR robot, see lect5 p.9-10 
        """
        #solve for intermediate values
        r = utils.dist(0,0,x,y) - 1e-12 #solves numerical issue at θ2 == 0 
        β = np.arccos((l1**2 + l2**2 - r**2) / (2*l1*l2)) #issues occures here
        γ = np.arccos((r**2 + l1**2 - l2**2) / (2*r*l1))
        α = np.arctan2(y,x)
        
        #handle different elbow states
        θ1 = α - γ
        θ2 = np.pi - β
        
        if elbow == "down":
            return θ1,θ2
        elif elbow == "up":
            θ1pm = θ1 + 2*γ  
            θ2pm = -θ2   
            return θ1pm,θ2pm

    
    def _IK_RRR_θa(self,x,y,θ3):
        """
        solves the IK problem given an ankle joint locked at angle θa 
        """
        #define local vars: 
        L1 = self.femur
        L2 = self.shank
        L3 = self.foot.len
        
        #determine the ankle state based on θ3  (consider refactoring into a property?)
        θ3 = utils.ROC(θ3,lower = 0,upper = 2*np.pi)
        if θ3 >= 0 and θ3 < np.pi:
            wrist = "down"
        if θ3 >= np.pi and θ3 <= 2*np.pi:
            wrist = "up"
    
        #determine Lf, the length of the ficticious segment
        a = np.pi - θ3
        Lf = (L2**2 + L3**2 - 2*(L2)*(L3)*np.cos(a))**.5
        
        #determine the IK for the RR robot formed by L1 and the fictitious Lf 
        θ1, θ2f = self._IK_RR(x,y,L1,Lf,"up")
        
        #determine the FK to the first link 
        x1 = L1*np.cos(θ1); y1 = L1*np.sin(θ1)
        
        #determine the IK from the tip of the first link, to the end effector location
        θ2, θ3 = self._IK_RR(x - x1,y - y1,L2,L3,wrist)
        θ2 -= θ1                    #angle correction for different coordinate frames 
        
        return θ1,θ2,θ3
        
    
    def IK(self,x,y,method = "jch"):
        """
        given a location x,y of the midfoot, what are the joint angles required to
        achieve that position? 
        """
        #check that IK can be performed
        ϵ = .001
        if utils.dist(0,0,x,y) + ϵ > self.femur + self.shank + self.foot.len:
            return np.nan,np.nan,np.nan
        
        #locked ankle
        elif method == "locked-ankle":
            θh,θk,θa = self._IK_RRR_θa(x,y,0)
            return θh,θk,θa
        
        #joint-coupling huristic
        elif method == "jch":
            neutral = self.foot.α
            θh,θk,θa = self._IK_RRR_θa(x,y,0)
            if not utils.noNans(θh,θk,θa):
                return np.nan,np.nan,np.nan
            hip2AnkleAngle = np.arctan2(y,x)
            self.ang = neutral + self.h0 + self.m_h * hip2AnkleAngle + self.k0 + self.m_k * θk
            self.ang = utils.clampAng(self.ang,self.ankle.θmin,self.ankle.θmax)
            θh,θk,θa = self._IK_RRR_θa(x,y,self.ang)
            return θh,θk,θa
        
            
        else:
            raise ValueError("method {} not yet implemented".format(method))

    
    def _FK(self,θh,θk,θa):
        """
        calculate the forward kinematics of each joint and the contact point
        returns: 
            vectors of x and y coordinates of forward kinematic
            [hip,knee,ankle,ball]
        """
        #calculate the sines and cosines of successive angles
        s_h   = np.sin(θh)        ; c_h   = np.cos(θh)
        s_hk  = np.sin(θh+θk)     ; c_hk  = np.cos(θh+θk)
        s_hka = np.sin(θh+θk+θa)  ; c_hka = np.cos(θh+θk+θa)
        
        #perform the forward kinematics 
        hip     = [0,0]
        knee    = [  hip[0] + self.femur*c_h ,      hip[1] + self.femur*s_h]                                     
        ankle   = [ knee[0] + self.shank*c_hk ,    knee[1] + self.shank*s_hk]
        ball    = [ankle[0] + self.foot.len*c_hka,ankle[1] + self.foot.len*s_hka]
        
        #reorganize into vectors
        x = [hip[0],knee[0],ankle[0],ball[0]]
        y = [hip[1],knee[1],ankle[1],ball[1]]
        
        return (x,y)
 
    
    def _Jac(self,θh,θk,θa,mode = "force"):
        """
        calculate the force or (velocity) jacobian from at a particular leg state
        by convention, jac is velocity jacobian jac_t is force jacobian, and 
        jinv_t is the inverse force space jacobian, etc. 
        """
        #the jacobian is expressable in terms of the forward kinematic quantities
        # duffy p.115
        xs,ys = self._FK(θh,θk,θa)
        xh,xk,xa,xb = xs[0],xs[1],xs[2],xs[3]
        yh,yk,ya,yb = ys[0],ys[1],ys[2],ys[3]
        
        
        #calculate the force jacobian 
        #               x(θ's)     y(θ's)    ϕ(θ's)
        jac = np.array([[-yb,        xb,       1],    #θh
                        [-(yb-yk),   xb-xk,    1],    #θk
                        [-(yb-ya),   xb-xa,    1]])   #θa
        
        if mode == "force":
            return jac
        elif mode == "velocity":
            return jac.T
        
        
    def reachableSpacePlot(self):
        """
        plot the outer reach of the leg,without regard to joint limits
        """
        t = np.linspace(0,2* np.pi,100)
        r = self.reach
        x = r * np.cos(t)
        y = r * np.sin(t)
        return x,y
    
         
    def Sample(self,samplingMode = "Cartesian", limits = []):
        """
        return a sampling of points, ([xs],[ys]) within the reachable space
        of the human model, subject to limit conditions that constrain the 
        sampled space
        inputs:
            sampling mode - FK, cartesian, polar
            limits - list of limiting fxn names, as strings.
        """
        
        if samplingMode == "FK":
            #calculate the set of points using sampling methods
            nh = 50; nk = 25; npts = nh * nk
            pts = np.zeros((nh,nk,2))
            θhs = np.linspace(self.hip.θmin,self.hip.θmax,nh)
            θks = np.linspace(self.knee.θmin,self.knee.θmax,nk)
            
            #sample the space - within joint constraints
            for i,θh in enumerate(θhs):
                for j,θk in enumerate(θks):
                    x,y = self._FK(θh,θk,self.ankle.θmin)
                    if self.limits(x,y,θh,θk,self.ankle.θmin,limits):
                        pts[i,j,:] = np.array((x[3],y[3]))

            #flatten pts (depending on if knee or hip is first)
            inner = "knee"
            if inner == "hip" : pts = pts.reshape(nh*nk,2,order = "F")
            if inner == "knee": pts = pts.reshape(nh*nk,2,order = "C")
        
        if samplingMode == "Cartesian":
            dx = .05   #x -sampling density
            dy = .05   #y -sampling density
            xs = np.arange(-self.reach,self.reach + dx,dx)
            ys = np.arange(-self.reach,self.reach + dy,dy)
            npts = xs.shape[0] * ys.shape[0]
            pts = np.zeros((xs.shape[0],ys.shape[0],2))
            
            #sample the space - within joint constraints
            for i,x in enumerate(xs):
                for j,y in enumerate(ys):
                    θh,θk,θa = self.IK(x,y)
                    if self.limits(x,y,θh,θk,θa,limits):
                        pts[i,j,:] = np.array((x,y))
                        
                        
        if samplingMode == "polar":
            pass
        
            
        #flatten pts (depending on if knee or hip is first)
        inner = "y" 
        if inner == "x" : pts = pts.reshape(npts,2,order = "F")
        if inner == "y":  pts = pts.reshape(npts,2,order = "C")
                                         
        #pop nans
        pts = pts[~np.isnan(pts).any(axis=1)]
        
        #cache list
        self.samplePoints = pts  #keep as a matrix? 

        #return
        return pts[:,0] , pts[:,1]
    
    
    
    def limits(self,x,y,θh,θk,θa,lims = []):
        """
        checks if a point satisfies the selected limit functions. 
        if it does, return True, else return false. 
        input:
            IK and FK vars
            lims - list of numbers, with indecies of which tests to run
        """
        #0 - can IK even be performed?
        if 0 in lims:
            θs = self.IK(x,y)
            if not utils.noNans(θs[0],θs[1],θs[2]):
                return False

        #1 - simple joint limits
        if 1 in lims:
            if not utils.between2Angles(θh,self.hip.θmin,self.hip.θmax) or \
               not utils.between2Angles(θk,self.knee.θmin,self.knee.θmax):
                return False
        
        #2 - alpha method - angle between hip and ankle joint
        if 2 in lims:
            xs,ys = self._FK(θh,θk,θa)
            α = np.arctan2(ys[2],xs[2])
            if not utils.between2Angles(α,self.αMin,self.αMax):
                return False
        
        #3 - combined joint limits
        if 3 in lims:
            pass
        
        return True
            
    def FK(self,θh,θk,θa):
        """
        given the joint angles θh,θk,θa, where is the middle of the foot located in space?
        """
        x,y = self._FK(θh,θk,θa) ; return (x[3],y[3])
    
    def FKplot(self,θh,θk,θa):
        """
        forward kinematic plotting vector for viewing human leg
        """
        _xs,_ys = self._FK(θh,θk,θa)
        ϕ = θh + θk + θa
        fxs,fys = self.foot.PlotPts(ϕ)
        fxs += _xs[2] ; fys += _ys[2]
        xs = np.hstack((np.array(_xs[:-1]),fxs))
        ys = np.hstack((np.array(_ys[:-1]),fys))
        return xs,ys
        
        
    def FS(self,x,y):
        """
        evaluate the Force Space of the human at a particular location location in the saggital plane
        and return a shape object that can be used to score FS overlap
        """
        θh,θk,θa, = self.IK(x,y)
        J_T = self._Jac(θh,θk,θa)
        J_invT = np.linalg.inv(J_T)
        cs = J_invT @  self.τs.corners(θh,θk,θa)
        fs = cs[0:2,:]
        return fs
    
    def FSpolygon(self,x,y):
        """
        generate the force space polygon
        """
    
    def FSplot(self,x,y,Scale):
        """
        given an endeffector location (x,y), return the xs and ys for plotting
        the force polygon
        """
        fs = self.FS(x,y) * Scale
        hull = ConvexHull(fs.T)
        verts = fs.T[hull.vertices,:]
        fs = verts.T
        xs = fs[0,:] ; ys = fs[1,:]
        return xs,ys
    
    
    def buildFScache(self):
        """
        run this function to build a cache (dictionary) of FS polygons which can be
        looked up via their x and y location with O(1) performance. 
        """
        pass
            

#----------------------------------- tests ----------------------------------
def test_human():
    h = Human()
    def test_FS():
        pass
    pass

test = False
if test:
    test_human()
    
        
#---------------------------------- plotting ---------------------------------
h = Human()
hfig = go.FigureWidget()

#configure layout of graph
hfig.update_xaxes(range=[-1.2*h.reach, 1.2*h.reach])
hfig.update_yaxes(range=[-1.2*h.reach, 1.2*h.reach])

side = 1000
hfig.update_layout(
    autosize=False,
    width=side + 100,
    height=side,
    legend=dict(x=.025, y=.975),
    margin=dict(l=80, r=80, t=0, b=80))


#initialize all plots present in visualization
hfig.add_scatter(line=dict(dash='dash',color='royalblue'), name = "workspace")
hfig.add_scatter(mode="markers", marker = dict(color = np.linspace(0,1,50*25)),name="Sample")
hfig.add_scatter(line=dict(color='royalblue'),name = "leg")
hfig.add_scatter(mode="markers", name = "forceSpace")
hfig.add_scatter(mode='markers', marker=dict(size=12,color="black"), name = "End Effector")
hfig.add_scatter(fill='toself', fillcolor='rgba(225, 0, 0, .2)', mode= 'none', name="force Space")
#hfig.add_scatter(mode="markers", marker = dict(color = np.linspace(0,1,8)),name="force Space")  


#setup UI
rng = h.reach; step = .01
layout=Layout(width='500px', height='20px')

workspace = Checkbox(value=True,description="workspace",dindent=False)
sample = Checkbox(value=False,description="sample",dindent=False)
forceSpace = Checkbox(value=False,description="forceSpace",dindent=False)

fsScale = FloatSlider(min=.25,max=3,step=step,value=1,description='Force Scale',layout=layout)
x1 = FloatSlider(min=-rng,max=rng,step=step,value=.75,description='x', layout=layout)
y1 = FloatSlider(min=-rng,max=rng,step=step,value=0,description='y', layout=layout)

h0 = FloatSlider(min=-np.pi,max=np.pi,step=step,value=h.h0,description='h0', layout=layout)
k0 = FloatSlider(min=-np.pi,max=np.pi,step=step,value=h.k0,description='k0', layout=layout)

m_h = FloatSlider(min=-1,max=1,step=step,value=h.m_h,description='m_h', layout=layout)
m_k = FloatSlider(min=-1,max=1,step=step,value=h.m_k,description='m_k', layout=layout)



#assemble UI
row1 = widgets.HBox([workspace,sample,forceSpace])
row2 = widgets.HBox([x1,fsScale])
row3 = widgets.HBox([y1])
row4 = widgets.HBox([h0,k0])
row5 = widgets.HBox([m_h,m_k])

ui = widgets.VBox([row1,row2,row3,row4,row5])

wdict = {"workspace":workspace,"sample":sample,"forceSpace":forceSpace,
         "x1":x1,"fsScale":fsScale,
         "y1":y1,
         "h0":h0,"k0":k0,
         "m_h":m_h,"m_k":m_k}

def update(workspace=True,sample=False,forceSpace=False,
           x1=1.5,fsScale=1,
           y1=0,
           h0=0,k0=0,
           m_h=0,m_k=0):
    
    
    with hfig.batch_update():
        
        #draw the workspace (eventually, just the boundry once that fxn is done)
        if workspace:
            xs,ys = h.reachableSpacePlot()
            hfig.data[0]['x'] = xs
            hfig.data[0]['y'] = ys
        else:
            hfig.data[0]['x'] = []
            hfig.data[0]['y'] = []
            
        #draw bounds Sample
        if sample:
            xs,ys = h.Sample("FK",[2])
            hfig.data[1]['marker']['color'] = np.linspace(0,1,h.samplePoints.shape[0])
            hfig.data[1]['x'] = xs
            hfig.data[1]['y'] = ys
        else:
            hfig.data[1]['x'] = []
            hfig.data[1]['y'] = []
            
        #set joint coupling huristic parameters
        h.h0 = h0
        h.k0 = k0      
        h.m_h = m_h
        h.m_k = m_k      
        
        #draw the human leg
        θh,θk,θa = h.IK(x1,y1,method = "jch")
        xs,ys = h.FKplot(θh,θk,θa)
        hfig.data[2]['x'] = xs
        hfig.data[2]['y'] = ys
        
        #draw the end effect location
        hfig.data[4]['x'] = [x1]
        hfig.data[4]['y'] = [y1]
        
    
    
        #draw the force space
        if forceSpace:
            xs,ys = h.FSplot(x1,y1,fsScale*.01)
            hfig.data[5]['x'] = xs + x1
            hfig.data[5]['y'] = ys + y1

#display interactive figure
out = widgets.interactive_output(update, wdict)
display(ui)
hfig

FigureWidget({
    'data': [{'line': {'color': 'royalblue', 'dash': 'dash'},
              'name': 'workspace'…

In [202]:
out

Output()

### plot the 3D tau space, with zero plane

In [348]:

#init τs figure

_fig = go.Figure(data=[go.Scatter3d(z=[0],x=[0],y=[0], name = "origin"),
                       go.Scatter3d(z=[0],x=[0],y=[0], name = "τSpace Prism"),
                       go.Mesh3d(z=[0],x=[0],y=[0], name="θh",opacity = .1, color='rgb(244,22,100)',delaunayaxis = 'x'),
                       go.Mesh3d(z=[0],x=[0],y=[0], name="θk",opacity = .1, color='rgb(244,22,100)',delaunayaxis = 'y'),
                       go.Mesh3d(z=[0],x=[0],y=[0], name="θa",opacity = .1, color='rgb(244,22,100)',delaunayaxis = 'z'),
                       go.Mesh3d(z=[0],x=[0],y=[0], name="τ_ϕ = 0",opacity=0.8),])



hτsfig = go.FigureWidget(_fig)
hτsfig.update_layout(title='human τSpace prism, with τ_ϕ = 0 cutting plane',
                     scene = dict(
                         xaxis_title='θh',
                         yaxis_title='θk',
                         zaxis_title='θa',
                         camera = dict([
                             ("up" , {'x': 0.900, 'y': -0.222, 'z': 0.373}),
                             ("eye", {'x': 1.05, 'y': 1.23, 'z': -1.77})]
                                     )
                                 ),
                     title_x=0.5,
                     autosize=False,
                     width=1200, height=800)


α = FloatSlider(min=0,max=1,step=.01,value=.0,description='α', layout=layout)
α = FloatSlider(min=0,max=1,step=.01,value=.0,description='α', layout=layout)


@interact(x1=x1,y1=y1,α=α)
def update(x1=1,y1=0,α=.1):  
    with hτsfig.batch_update():
        
        #draw τs rectangular prism
        θh,θk,θa = h.IK(x1,y1,method = "jch")
        xs,ys,zs = h.τs.prismPlot(θh,θk,θa)
        hτsfig.data[1]['x']= xs
        hτsfig.data[1]['y']= ys
        hτsfig.data[1]['z']= zs
        
        #draw principle planes
        xs,ys,zs = h.τs.θhPlanePlot()
        hτsfig.data[2]['x']= xs
        hτsfig.data[2]['y']= ys
        hτsfig.data[2]['z']= zs
        
        xs,ys,zs = h.τs.θkPlanePlot()
        hτsfig.data[3]['x']= xs
        hτsfig.data[3]['y']= ys
        hτsfig.data[3]['z']= zs
        
        xs,ys,zs = h.τs.θaPlanePlot()
        hτsfig.data[4]['x']= xs
        hτsfig.data[4]['y']= ys
        hτsfig.data[4]['z']= zs
        
        #set plane visability
        hτsfig.data[2]['opacity'] = α
        hτsfig.data[3]['opacity'] = α
        hτsfig.data[4]['opacity'] = α
        
        
        #draw intersection plane
        jac_t = h._Jac(θh,θk,θa)
        n_p = h.τs.n_p(jac_t)
        xs,ys,zs = h.τs.polygonPlot(θh,θk,θa,n_p)
        hτsfig.data[5]['x']= xs
        hτsfig.data[5]['y']= ys
        hτsfig.data[5]['z']= zs
        
        
        
hτsfig.update_traces()

interactive(children=(FloatSlider(value=0.75, description='x', layout=Layout(height='20px', width='500px'), ma…

FigureWidget({
    'data': [{'name': 'origin',
              'type': 'scatter3d',
              'uid': '9c778e…

In [272]:
θh,θk,θa = h.IK(x1.value,y1.value,method = "jch")
jac_t = h._Jac(θh,θk,θa)
n_p = h.τs.n_p(jac_t)
h.τs.planeCut(θh,θk,θa,n_p)



array([1., 1., 1.])

## discussion points


# to do: 

- determine how to limit the the torque space to only the torque space which doesn't produce end effector torque
- solve the problem with the IK dropout - start by looking with a debugger
- develop the spline function to scale the torque space rectangular prism


















#### to do (immediate next steps):
- should we limit ourselves to solutions with zero torque at the end effector, as that's what the haptic system will do? - this should form a cut plane through the torque space cube, that results in a parellelogram geometry once again. 
- finish sampling functions       X
- make a condition number viewer  
- visualize the asymmetric force space X
- tune huristic function based on literature values / observations
- work on spline functions for actual joint limits (fast splines)

if there is time... 
- 
- consider making an importance weighting function on the optimizer - something which is a regularization term which says "a jacobian match near the center of the workspace is more important than one at the edge of the workspace" the way I was thinking about implementing this is a normal distribution shaped importance function, with distance from a mean line.
- consider a different type of sampling, polar uniform sampling, and call the other type of sampling "FK" sampling 

what are my goals for this visualizer?
* be able to debug the human class
* visualize the bounds and bounds check function
* visualize the process of IK, and how it's being solved
* visualize the force space of the human


#bugs:
- IK with mobile ankle flickers (doesn't solve) in some internal locations. figure out why. 

notes:
* the calculated space depends on whether the robot's are elbow in or elbow out.
* $det(A^{-1}) = \frac{1}{det(A)}$


### references:
- for calculating the intersection of a plane and a rectangular prism: https://www.cg.informatik.uni-siegen.de/data/Publications/2005/rezksalamaVMV2005.pdf
- for anthropometric measurements of the segment lengths
- for individual joint limits
- for compound joint limits
- for static joint torques as a function of joint angles
    - a
    - b
    - c
- for jacobian from the forward kinematic [Duffy 1996]

<a id="joint_exp"></a>
## Joint exploration - put the two together [ &#x21ea;](#top)


* when visualize forceSpaces is selected, you have free control over the movement of \theta1 and \theta2, and can see the calculated force spaces. FR scales the robot force space. when you select that button, the blue robot becomes bold, and is the leader, the robot follows through IK. if the human leaves the area, then robot should disappear, until we are back in the interaction area. 
* the feature is separate from and is drawn over top of the mode controur map. 

<a id="solution"></a>
## Solve the constrained optimization problem [ &#x21ea;](#top)

* perhaps explore 2 or more methods of optimization, and see that they converge on the same solution. 
* perhaps there should be a specific penalty against singularity, besides just allowing the overlap to be zero?
* optimizer object should contain a reference to R and H, and should be responsible for the "scoring function" while the robot and human objects are responsible for returning the shape objects. 


<a id="notes"></a>
## notes:

speed up code with cython
https://jakevdp.github.io/blog/2017/12/11/live-coding-cython-ising-model/?fbclid=IwAR1_KPRJACFUJk6-TYb1DqumE9Cn3hk40QUOl10UCyItL2z4Ft5x_s-Nc60
numba, JIT

python optimization with pyomo http://egon.cheme.cmu.edu/ewo/docs/EWO_Seminar_03_10_2017.pdf
